In [1]:
import os

In [6]:
%pwd

'e:\\ETE_Disease_Classification'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\ETE_Disease_Classification'

In [5]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model('artifacts/training/model.keras')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    evaluation: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from src.constants import *
from src.utils.common import read_yaml,create_dir,save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dir([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = self.config.training.trained_model_path,
            training_data = os.path.join(self.config.data_ingestion.unzip_dir,'Chicken-fecal-images'),
            evaluation = self.config.evaluation.root_dir,
            all_params= self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config = EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1.0/255.0,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {'loss': self.score[0],'accuracy':self.score[1]}
        directory = self.config.evaluation
        os.makedirs(directory,exist_ok=True)
        save_json(path=Path(os.path.join(directory, 'scores.json')), data=scores)

In [16]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-05-29 00:33:07,372: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-29 00:33:07,379: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-29 00:33:07,384: INFO: common: created a directory at: artifacts]


Found 78 images belonging to 2 classes.


e:\ETE_Disease_Classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.2630 - loss: 32.3627
[2024-05-29 00:33:31,110: INFO: common: json file saved at artifacts\evaluation\scores.json]
